In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_date, year, lit,
    when, trim, regexp_replace
)
from pyspark.sql.types import (
    IntegerType, DoubleType, StringType, DateType
)

In [0]:
nf.printSchema()

In [0]:
nf_corrigido.write.mode("overwrite").saveAsTable("classes.silver.notas_fiscais_corrigida")

In [0]:
display(nf_corrigido)

In [0]:
from pyspark.sql.functions import col, to_date, trim, initcap, year, month, round

nf = spark.sql("select * from classes.bronze.notas_fiscais")

column_mapping = {
    "nr_nota_fiscal": "invoice_id",
    "nr_linha_nf": "invoice_line_id",
    "nr_pedido_referencia": "order_id",
    "nome_vendedor": "salesperson_name",
    "data_emissao_nf": "invoice_date",
    "data_vencimento_nf": "due_date",
    "cod_produto": "product_code",
    "nome_produto": "product_name",
    "qtde_produto": "product_quantity",
    "valor_total_produto": "total_line_value",
    "imposto_produto": "tax_amount"
}

for old_col, new_col in column_mapping.items():
    nf = nf.withColumnRenamed(old_col, new_col)

nf = nf.withColumn("invoice_date", to_date(col("invoice_date"))) \
       .withColumn("due_date", to_date(col("due_date"))) \
       .withColumn("salesperson_name", trim(initcap(col("salesperson_name")))) \
       .withColumn("product_name", trim(initcap(col("product_name")))) \
       .withColumn("tax_amount", round(col("tax_amount"), 2)) \
       .dropDuplicates(["invoice_id", "invoice_line_id"]) 


nf = nf.withColumn("invoice_year", year(col("invoice_date"))) \
       .withColumn("invoice_month", month(col("invoice_date")))


nf.write.mode("overwrite") \
    .partitionBy("invoice_year", "invoice_month") \
    .saveAsTable("classes.silver.notas_fiscais")

display(nf)

In [0]:

nf.write.mode("overwrite") \
    .partitionBy("invoice_year", "invoice_month") \
    .saveAsTable("classes.silver.notas_fiscais")